In [1]:
using JSON
using Random
using Distributions
using MySQL
using DotEnv
using DataFrames
using PrettyTables
cfg = DotEnv.config()
const DB = DBInterface
include("/home/peters/code/mapinator/estimation/functions.jl")
YEAR_INTERVAL = 2003:2023  # change this to select the years of data to include in the estimation
NUM = 5 # variable set it here
NUMSINKS = 6 #4 sinks are assumed in some of the code
sinks = ["Other Academic","Government","Private Sector","Teaching Universities","Postdoc", "Sessionals","Column Totals"]
function name_to_oid(name, institution_mapping)
    for (key, value) in institution_mapping
        if value == name
            return key
        end
    end
    return "not found"
end

name_to_oid (generic function with 1 method)

bucket_extract

In [2]:
 academic_builder, sink_builder, academic, acd_sink, gov_sink, pri_sink, tch_sink, postdoc, lecturer, institution_mapping = functions.process_data(YEAR_INTERVAL);

Debug: 14875 returned from api
Debug: 7272 academic placements
Debug: 7603 sinks
Debug: 14875 in all


In [3]:
println(length(academic_builder), " total assistant professor outcomes")
    println(length(sink_builder), " other outcomes")
    sample_size = length(academic_builder)+length(sink_builder)
    println(sample_size, " total outcomes")
println(length(acd_sink), " Other academic institutions")
println(length(gov_sink), " Government institutions")
println(length(pri_sink), " Private Sector institutions")
println(length(tch_sink), " Teaching institutions")
println(length(postdoc), " Institutions hiring postdocs")
println(length(lecturer), " Institutions hiring sessionals")


7272 total assistant professor outcomes
7603 other outcomes
14875 total outcomes
552 Other academic institutions
156 Government institutions
195 Private Sector institutions
627 Teaching institutions
515 Institutions hiring postdocs
366 Institutions hiring sessionals


In [4]:
institutions, out = functions.create_adjacency_matrix(academic_builder, sink_builder, academic,acd_sink, gov_sink, pri_sink, tch_sink, postdoc, lecturer);

In [5]:
est_obj, est_alloc = functions.doit(out, collect(academic),collect(acd_sink), collect(gov_sink), collect(pri_sink), collect(tch_sink), collect(postdoc), collect(lecturer),institutions, NUM, NUMSINKS);

In [6]:
est_mat, est_count = functions.bucket_extract(est_alloc, out, NUM, NUMSINKS)
M = est_mat

11×5 Matrix{Int32}:
 219  326  442  630   35
  95  256  620  564   27
  15   43  703  104    2
  37  102  660  317   11
  61   61   20   59   67
 317  355  416  404  220
 117  247  545  441   48
 103  202  672  388   34
 274  452  380  555  135
 369  560  559  574  229
 164  169  141  210  119

In [7]:
t_table, t_allocation = functions.tiered_allocation(M, est_count, institutions, est_alloc, NUM, NUMSINKS);
t_table

11×5 Matrix{Int32}:
 703  104   43   15    2
 660  317  102   37   11
 620  564  256   95   27
 442  630  326  219   35
  20   59   61   61   67
 416  404  355  317  220
 545  441  247  117   48
 672  388  202  103   34
 380  555  452  274  135
 559  574  560  369  229
 141  210  169  164  119

In [8]:
table_latex = functions.nice_table(t_table, NUM, NUMSINKS, sinks)
table_latex

┌───────────────────────┬────────┬────────┬────────┬────────┬────────┬────────────┐
│                       │ Tier 1 │ Tier 2 │ Tier 3 │ Tier 4 │ Tier 5 │ Row Totals │
├───────────────────────┼────────┼────────┼────────┼────────┼────────┼────────────┤
│                Tier 1 │    703 │    104 │     43 │     15 │      2 │        867 │
│                Tier 2 │    660 │    317 │    102 │     37 │     11 │       1127 │
│                Tier 3 │    620 │    564 │    256 │     95 │     27 │       1562 │
│                Tier 4 │    442 │    630 │    326 │    219 │     35 │       1652 │
│                Tier 5 │     20 │     59 │     61 │     61 │     67 │        268 │
│        Other Academic │    416 │    404 │    355 │    317 │    220 │       1712 │
│            Government │    545 │    441 │    247 │    117 │     48 │       1398 │
│        Private Sector │    672 │    388 │    202 │    103 │     34 │       1399 │
│ Teaching Universities │    380 │    555 │    452 │    274 │    135 │      

In [9]:
d = DB.connect(MySQL.Connection,cfg["host"], cfg["user"], cfg["password"], db =cfg["database"], port = parse(Int64,cfg["port"]));
query = "select i.name,j.institution_id ,
SUM(CASE
  WHEN j.estimated_tier = 1 THEN count ELSE 0 END
) AS TIER1,
SUM(CASE
  WHEN j.estimated_tier = 2 THEN count ELSE 0 END
) AS TIER2,
SUM(CASE
  WHEN j.estimated_tier = 3 THEN count ELSE 0 END
) AS TIER3,
SUM(CASE
  WHEN j.estimated_tier = 4 THEN count ELSE 0 END
) AS TIER4,
SUM(CASE
  WHEN j.estimated_tier = 5 THEN count ELSE 0 END
) AS TIER5


from ftrecruiter.institutions i join
(select d.institution_id, d.estimated_tier, count(*) as count from bootstrap_data d left join bootstrap_trials t  
on t.trial_id=d.trial_id where t.number_of_tiers=?
group by institution_id, estimated_tier) j on i.institution_id=j.institution_id where j.institution_id=?
group by j.institution_id"
get_statement =  DB.prepare(d, query);


In [31]:
function get_confidence(tiers, institution_id, get_statement)
    m = DB.execute(get_statement, [tiers, institution_id]) |> DataFrame
    try
        r = first(m)
        t = 0
        s = 0
        s+= r.TIER1
        s+= r.TIER2
        s+= r.TIER3
        s+= r.TIER4
        s+= r.TIER5
        est = [r.TIER1/s, r.TIER2/s, r.TIER3/s, r.TIER4/s, r.TIER5/s]
        return findmax(est)
    catch
        return "Not found"
    end
end

get_confidence (generic function with 1 method)

In [39]:
get_confidence(NUM, name_to_oid("University of Wisconsin, Madison", institution_mapping), get_statement)

(0.9830508474576271, 1)

In [41]:
for j in 1:NUM
    println("Tier $j:")
    tier = Dict{}()
    for (i, type) in enumerate(t_allocation)
        if type == j
            push!(tier, (institutions[i] => get_confidence(NUM, name_to_oid(institutions[i], institution_mapping), get_statement)))
           # println("  ", institutions[i], " Confidence: ", get_confidence(NUM, name_to_oid(institutions[i], institution_mapping), get_statement))
        end
    end
    pretty_table(tier; header = ["University", "Confidence"])
    pretty_table(tier; header = ["University", "Confidence"], backend=Val(:latex))
end

Tier 1:
┌──────────────────────────────────────────────────┬─────────────────────────┐
│                                             Keys │                  Values │
│                                              Any │                     Any │
├──────────────────────────────────────────────────┼─────────────────────────┤
│                                  Yale University │ (0.9830508474576271, 1) │
│                              Columbia University │ (0.9830508474576271, 1) │
│                               Harvard University │ (0.9830508474576271, 1) │
│                                  Duke University │ (0.9830508474576271, 1) │
│            Massachusetts Institute of Technology │ (0.9661016949152542, 1) │
│               University of California, Berkeley │ (0.9830508474576271, 1) │
│                 University of Wisconsin, Madison │ (0.9830508474576271, 1) │
│ London School of Economics and Political Science │ (0.9830508474576271, 1) │
│                       University of Pennsy